In [2]:
!pip install --upgrade pip

In [3]:
!pip install mediapipe
!pip install opencv-python
!pip install tensorflow

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from threading import Thread
import time
import tensorflow as tf



2024-10-01 19:07:12.269297: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 19:07:12.290843: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 19:07:12.468009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 19:07:12.661848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 19:07:12.829440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

OSError: SavedModel file does not exist at: path_to_your_model/{saved_model.pbtxt|saved_model.pb}

In [5]:
# Load your model
model = tf.saved_model.load('path_to_your_model')

# Print model signatures
print(model.signatures)

# Initialize video capture object
cap = cv2.VideoCapture('WhatsApp Video 2024-09-18 at 10.18.25 AM.mp4')

# Check if video is opened successfully
if not cap.isOpened():
    print("Error: Video file could not be opened.")
    exit()

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize MediaPipe drawing utils
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points using tangent


OSError: SavedModel file does not exist at: path_to_your_model/{saved_model.pbtxt|saved_model.pb}

In [6]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    # Vector differences
    ab = a - b
    bc = c - b

    # Calculate the angle using arctan2 for better accuracy
    angle = np.arctan2(np.linalg.norm(np.cross(ab, bc)), np.dot(ab, bc))
    return np.degrees(angle)

# Lists to store angles and timestamps for plotting
left_knee_angles = []
right_knee_angles = []
timestamps = []

# Set desired speed-up factor (e.g., 2 for double speed)
speed_up_factor = 2
fps = cap.get(cv2.CAP_PROP_FPS)  # Get original frames per second
new_fps = fps * speed_up_factor  # Increase frame rate

# Set up the plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.set_title('Left Knee Angle over Time')
ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Angle (degrees)')
ax2.set_title('Right Knee Angle over Time')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Angle (degrees)')

# Create lines for plotting
line1, = ax1.plot([], [], label='Left Knee Angle', color='blue')
line2, = ax2.plot([], [], label='Right Knee Angle', color='red')

# Set limits for plots
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 180)
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 180)

# Function to initialize the plot
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return line1, line2

# Function to update the plot
def update_plot(frame):
    # Update the plot data
    line1.set_data(timestamps, left_knee_angles)
    line2.set_data(timestamps, right_knee_angles)
    
    # Adjust plot limits if necessary
    if len(timestamps) > 0:
        ax1.set_xlim(max(0, timestamps[-1] - 10), timestamps[-1] + 1)
        ax2.set_xlim(max(0, timestamps[-1] - 10), timestamps[-1] + 1)
    
    return line1, line2



NameError: name 'cap' is not defined

In [7]:
# Function to process the video and update angles
def process_video():
    global frame_count
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break

        # Convert the frame from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Set image as non-writable to improve performance
        image.flags.writeable = False
        
        # Process the image and detect pose
        results = pose.process(image)
        
        # Set image as writable again
        image.flags.writeable = True
        
        # Convert back to BGR for displaying
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Extract coordinates of relevant landmarks for left leg
            landmarks = results.pose_landmarks.landmark
            hip_left = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
            knee_left = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y)
            ankle_left = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
            
            # Extract coordinates of relevant landmarks for right leg
            hip_right = (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
            knee_right = (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)
            ankle_right = (landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y)
            
            # Calculate the angles
            angle_left = calculate_angle(hip_left, knee_left, ankle_left)
            angle_right = calculate_angle(hip_right, knee_right, ankle_right)
            
            # Store the angles and timestamp
            left_knee_angles.append(angle_left)
            right_knee_angles.append(angle_right)
            timestamps.append(frame_count / new_fps)  # Convert frame number to seconds
            
            # Display the angles on the image
            cv2.putText(image, f'Left Knee Angle: {angle_left:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'Right Knee Angle: {angle_right:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the frame
        cv2.imshow('Pose Detection', image)
        
        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1

    # Release resources
    cap.release()
    cv2.destroyAllWindows()



In [8]:
# Start video processing thread
frame_count = 0
video_thread = Thread(target=process_video)
video_thread.start()

# Create the animation
ani = FuncAnimation(fig, update_plot, frames=range(0, len(timestamps)), init_func=init, blit=True, interval=1000 / new_fps)

plt.show()


Exception in thread Thread-5 (process_video):
Traceback (most recent call last):
  File "/home/pranith/miniconda3/envs/myenv/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/pranith/miniconda3/envs/myenv/lib/python3.12/threading.py", line 1010, in run


NameError: name 'fig' is not defined

    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_121982/554065004.py", line 4, in process_video
NameError: name 'cap' is not defined. Did you mean: 'map'?
